In [1]:
import numpy as np
import urllib
from sklearn import preprocessing
import pandas as pd
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
from numpy import loadtxt, where
from pylab import scatter, show, legend, xlabel, ylabel
from matplotlib.colors import ListedColormap
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC


In [2]:
#Preprocessed clean version of input file
input_file = "Cleaned_file.csv"

In [3]:
df = pd.read_csv(input_file)

In [4]:
# Printing Data Shape
df.shape

(11670, 6)

In [5]:
df.head(10)

,District,Date,Primary Type,Arrest,Crime Weightage,Label
0,11,5/3/16 21:00,NARCOTICS,True,3,1
1,8,5/3/16 21:35,THEFT,False,1,1
2,9,5/3/16 14:45,ASSAULT,False,3,1
3,8,5/3/16 18:42,BATTERY,False,2,1
4,9,5/3/16 19:30,CRIMINAL DAMAGE,False,3,1
5,11,5/3/16 13:30,OTHER OFFENSE,False,2,0
6,1,5/3/16 19:30,THEFT,False,1,1
7,1,5/3/16 9:11,DECEPTIVE PRACTICE,False,1,0
8,1,5/3/16 6:30,THEFT,False,1,1
9,16,5/3/16 18:30,THEFT,False,1,0


In [6]:
#dataset = np.loadtxt(input_file, delimiter=",")

In [7]:
# Giving numeric values for text data
primaryType = preprocessing.LabelEncoder()
arrest = preprocessing.LabelEncoder()
#date = preprocessing.LabelEncoder()

In [8]:
df["Primary Type"] = primaryType.fit_transform(df["Primary Type"])
df["Arrest"] = arrest.fit_transform(df["Arrest"])
#df["Date"] = date.fit_transform(df["Date"])

In [9]:
df['Date'] = df['Date'].apply(lambda x: x.split('/')[0])

In [10]:
df.head(10)

,District,Date,Primary Type,Arrest,Crime Weightage,Label
0,11,5,16,1,3,1
1,8,5,28,0,1,1
2,9,5,1,0,3,1
3,8,5,2,0,2,1
4,9,5,6,0,3,1
5,11,5,22,0,2,0
6,1,5,28,0,1,1
7,1,5,8,0,1,0
8,1,5,28,0,1,1
9,16,5,28,0,1,0


In [11]:
#df["Date"] = date.inverse_transform(df["Date"])
#encod = preprocessing.OneHotEncoder()
#dataMat =  encod.fit_transform(df)
#print(np.matrix(dataMat))

In [12]:
#df.head()
keep_col = ['District', 'Date', 'Primary Type', 'Crime Weightage', 'Label']
new_df = df[keep_col]

In [13]:
array = new_df.values
array

array([[11, '5', 16, 3, 1],
       [8, '5', 28, 1, 1],
       [9, '5', 1, 3, 1],
       ..., 
       [9, '10', 10, 4, 0],
       [2, '10', 10, 4, 0],
       [2, '5', 10, 4, 0]], dtype=object)

In [14]:
#columns = ["District", "Date", "Primary Type", "Arrest", "Crime Weightage"]

In [15]:
# Features
X = list(array[:,0:4])

In [16]:
# Target
Y = list(array[:,4])

In [17]:
totalTarget = df["Label"].count()

# Count of 0s in the target
zeroCount = df[(df["Label"] == 0)].count()[0]

# Count of 1s in the target
oneCount = df[(df["Label"] == 1)].count()[0]

#Probability of 0s in the target
probZero = zeroCount / totalTarget

#Probability of 1s in the target
probOne = oneCount / totalTarget

print("Probability of 0s in the Target ", '%0.3f'%(probZero))
print("Probability of 1s in the Target ", '%0.3f'%(probOne))

Probability of 0s in the Target  0.589
Probability of 1s in the Target  0.411


#### Since probability for 0s in the target is higher than 1s, we'll use 0.59 as out baseline accuracy.

#### Splitting Train and Test Data 

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.70, random_state=0)

#### So below we can see that Logistic Regression accuracy is best when argument <i>C >= 0.01</i> i.e. 0.68169

In [19]:
# Calculating accuracy of LOGISTIC REGRESSION using CROSS_VAL_SCORE
C= [5,2,1,0.1,0.01,0.001,0.002,0.0001,0.00001]
for c in C: 
    clf = LogisticRegression(C = c, max_iter = 1000, random_state=0, solver='sag')
    scoresLR = cross_val_score(clf,X_train,y_train,cv=10,scoring='accuracy')
    print("Accuracy with C = ",c,": %0.5f(+/- %0.5f)" %(scoresLR.mean(), scoresLR.std()*2))

Accuracy with C =  5 : 0.68058(+/- 0.02743)
Accuracy with C =  2 : 0.68058(+/- 0.02743)
Accuracy with C =  1 : 0.68058(+/- 0.02743)
Accuracy with C =  0.1 : 0.68132(+/- 0.02742)
Accuracy with C =  0.01 : 0.68169(+/- 0.03421)
Accuracy with C =  0.001 : 0.68132(+/- 0.03117)
Accuracy with C =  0.002 : 0.68144(+/- 0.03282)
Accuracy with C =  0.0001 : 0.68144(+/- 0.03151)
Accuracy with C =  1e-05 : 0.63100(+/- 0.02456)


#### Best Accuracy with Decision Tree Classifier is 0.99951

In [20]:
# Calculating accuracy of DECISION TREE CLASSIFIER using CROSS_VAL_SCORE
dtree = tree.DecisionTreeClassifier()
scoresDT = cross_val_score(dtree,X_train,y_train,cv=10,scoring='accuracy')
print("Accuracy with Decision Tree: %0.5f (+/- %0.5f)" % (scoresDT.mean(), scoresDT.std() * 2))

Accuracy with Decision Tree: 0.99976 (+/- 0.00098)


#### Best Accuracy with Naive Bayes is 0.69467

In [21]:
# Calculating accuracy of NAIVE BAYES using CROSS_VAL_SCORE
gnb = GaussianNB()
scores = cross_val_score(gnb,X_train,y_train,cv=10,scoring='accuracy')
print("Accuracy with Gaussian Naive Bayes: %0.5f (+/- %0.5f)" % (scores.mean(), scores.std() * 2))

Accuracy with Gaussian Naive Bayes: 0.69467 (+/- 0.03100)


#### Best Accuracy with MLPC Classifier is when <i>learning_rate = 0.003</i> i.e. 0.81758

In [ ]:
L = [0.001,0.002,0.003,0.004,0.005,0.006,0.0001,0.00001, 0.000001]
func = ['logistic', 'tanh', 'relu']
hLayer = [1, 2, 3]
neurons = [3, 4, 5, 6, 7]
alpha = [0.1, 0.001, 0.0001, 0.00001]

for a in alpha:
    for l in L:
        for f in func:
            for h in hLayer:
                for n in neurons:
                    temp = []
                    for i in range(h):
                        temp.append(n)
                    temp = tuple(temp)
                    MLPC = MLPClassifier(random_state=0,hidden_layer_sizes=temp, learning_rate='adaptive',max_iter=10000, learning_rate_init=l, alpha=a, activation=f) 
                    scoresMLPC = cross_val_score(MLPC,X_train,y_train,cv=10,scoring='accuracy')
                    print("Accuracy with MLPC Classifier with L=",l,"a=", a, "hl=", temp, "func=", f, ": %0.5f (+/- %0.5f)" % (scoresMLPC.mean(), scoresMLPC.std() * 2))


Accuracy with MLPC Classifier with L= 0.001 a= 0.1 hl= (3,) func= logistic : 0.69638 (+/- 0.02501)


#### Best Accuracy with Support Vector Machines is 0.96523

In [ ]:
param = ['kernel', 'C', 'degree']
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
C = [0.1, 1, 10]
degree = [2, 3, 4]

for k in kernel:
    for c in C:
        for d in degree:
            svclf = SVC(cache_size=1000, kernel=k,C=c,degree=d) # Increased Cache size for larger datasets
            scores = cross_val_score(svclf,X_train,y_train,cv=10,scoring='accuracy')
            print("Accuracy with SVM parameters:K=",k," C=", c, " d=", d, ": %0.5f (+/- %0.5f)" % (scores.mean(), scores.std() * 2))

#### Accuracy of the various models in the order of increasing accuracy:
1. Logistic Regression     : 0.61068
2. Naive Bayes             : 0.63920
3. MLPC Classifer          : 0.85161
4. Support Vector Machines : 0.96523
5. Decision Tree Classifier: 0.99951


#### So comparing the accuracy for all the models, we found Support Vector Machines to be the best for our dataset as it gives the best accuracy of  0.96523. But since SVM and MLPC models runs very slow on Large Datasets, we preferred to use  

In [ ]:
'''Y_pred    = clf.predict(X_test)
Y_score   = clf.decision_function(X_test)
accuracy  = metrics.accuracy_score(y_test, Y_pred)
precision = metrics.precision_score(y_test, Y_pred)
recall    = metrics.recall_score(y_test, Y_pred)
AUC       = metrics.roc_auc_score(y_test, Y_score)'''